[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artbert/VoiceChatLLM/blob/main/Voice_LLM_Chat_Colab.ipynb)

![Runtime: Colab](https://img.shields.io/badge/runtime-Colab-4285F4?style=for-the-badge&logo=googlecolab&logoColor=white)

## Setup Instructions

This notebook contains the code for a voice-enabled LLM chat application. To run this application, you need to set up your environment by installing the necessary libraries and downloading the required models.

### Prerequisites

*   **Google Colab Environment:** This notebook is designed to run in a Google Colab environment.
*   **Python 3:** Ensure you have Python 3 installed.

### Library Installation

The required Python libraries are installed using `pip`. The following cell in the notebook handles these installations:

```python
!pip3 install piper-tts -q
!pip3 install ffmpeg-python -q
!pip3 install vosk -q
```

Simply run this cell to install the necessary packages.

### Model Downloads

This application requires a Piper voice model for Text-to-Speech (TTS) and a Vosk speech recognition model for Speech-to-Text (STT), as well as a Large Language Model (LLM).

The Vosk speech recognition model (`vosk-model-en-us-0.22-lgraph`) is automatically downloaded by the `vosk.Model` constructor if not already present.

The LLM model (`Gensyn/Qwen2.5-0.5B-Instruct`) is automatically downloaded and loaded using the `transformers` library. This is a very lightweight LLM language model that **DOES NOT** require a GPU environment.

Run the respective code cells in the notebook to download and load these models.

### Running the Application

Once the libraries are installed and models are downloaded, you can run the remaining code cells in the notebook sequentially to start and interact with the voice chat application.

## Usage Instructions

Once the setup is complete and the models are loaded, you can run the application cells to start the voice chat interface.

### Starting the Application

Run the code cell directly following the "Starting The Application" markdown heading. This cell initializes and displays the chat interface.

### Interacting with the Application

After running the start cell, a chat interface will appear.
*   **Voice Input:** Look for a microphone button and click it to start speaking. Speak clearly and concisely. Click the STOP button to stop recording. Your spoken input will be transcribed into text.
*   **Text Input:** You can also typically type your message into a text box provided in the interface and press Enter or click a send button.
*   **Receiving Responses:** The application will process your input using the LLM. The response will be displayed as text in the chat interface. If the Text-to-Speech model is working correctly, you will also hear the response spoken aloud.

### Stopping the Application

To gracefully stop the application and release resources, run the code cell directly following the "Stopping The Application" markdown heading.

### Libraries Installation

In [ ]:
import warnings
warnings.filterwarnings("ignore")

!pip3 install piper-tts -q
!pip3 install ffmpeg-python -q
!pip3 install vosk -q

### Piper Voices Download

In [ ]:
# Create an application folder.
!mkdir voice_llm_chat
%cd voice_llm_chat

# Let's download some nice Piper voices

!wget -q -O en_US-danny-low.onnx https://huggingface.co/rhasspy/piper-voices/resolve/v1.0.0/en/en_US/danny/low/en_US-danny-low.onnx?download=true
!wget -q -O en_US-danny-low.onnx.json https://huggingface.co/rhasspy/piper-voices/resolve/v1.0.0/en/en_US/danny/low/en_US-danny-low.onnx.json?download=true

!wget -q -O en_US-amy-medium.onnx https://huggingface.co/rhasspy/piper-voices/resolve/main/en/en_US/amy/medium/en_US-amy-medium.onnx?download=true
!wget -q -O en_US-amy-medium.onnx.json https://huggingface.co/rhasspy/piper-voices/resolve/main/en/en_US/amy/medium/en_US-amy-medium.onnx.json?download=true

!wget -q -O en_US-hfc_male-medium.onnx https://huggingface.co/rhasspy/piper-voices/resolve/main/en/en_US/hfc_male/medium/en_US-hfc_male-medium.onnx?download=true
!wget -q -O en_US-hfc_male-medium.onnx.json https://huggingface.co/rhasspy/piper-voices/resolve/main/en/en_US/hfc_male/medium/en_US-hfc_male-medium.onnx.json?download=true

!wget -q -O en_US-lessac-medium.onnx https://huggingface.co/rhasspy/piper-voices/resolve/main/en/en_US/lessac/medium/en_US-lessac-medium.onnx?download=true
!wget -q -O en_US-lessac-medium.onnx.json https://huggingface.co/rhasspy/piper-voices/resolve/main/en/en_US/lessac/medium/en_US-lessac-medium.onnx.json?download=true

!wget -q -O en_US-ryan-medium.onnx https://huggingface.co/rhasspy/piper-voices/resolve/main/en/en_US/ryan/medium/en_US-ryan-medium.onnx?download=true
!wget -q -O en_US-ryan-medium.onnx.json https://huggingface.co/rhasspy/piper-voices/resolve/main/en/en_US/ryan/medium/en_US-ryan-medium.onnx.json?download=true


piper_voices = {"amy": "en_US-amy-medium.onnx", "danny": "en_US-danny-low.onnx", "hfc_male": "en_US-hfc_male-medium.onnx", "lessac": "en_US-lessac-medium.onnx", "ryan": "en_US-ryan-medium.onnx"}


### Libraries Import

In [3]:
import time
import sys
import json
from IPython.display import HTML, display
from vosk import Model, KaldiRecognizer
from base64 import b64decode
from google.colab import output
import ffmpeg
import threading
import ipywidgets as widgets
import IPython
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from piper.voice import PiperVoice

### Download Voice LLM Chat Scripts

You can check the source code of the program on GitHub at [this](https://github.com/artbert/VoiceChatLLM) link.

In [4]:
!wget -q https://raw.githubusercontent.com/artbert/VoiceChatLLM/refs/heads/main/utils/voice_llm_chat.py
!wget -q https://raw.githubusercontent.com/artbert/VoiceChatLLM/refs/heads/main/utils/voice_llm_chat_frontend.py

In [5]:
from voice_llm_chat import VoiceLLMChatBackend
from voice_llm_chat_frontend import VoiceLLMChatFrontend_Colab

### Configuration Variables

In [6]:
# Choose the system message that best meets your needs.
llm_model_system_message = "You are a supportive voice assistant that replies with one or two brief sentences. Your replies should avoid any text formatting."

# You can test various parameter configurations.
llm_model_temperature = 0.1
llm_model_max_tokens = 256
llm_model_top_k = 100
llm_model_top_p = 1

### Loading Models

In [ ]:
# Load the Piper voice model
chosen_piper_voice = piper_voices["lessac"]
try:
    voice_model = PiperVoice.load(chosen_piper_voice)
except FileNotFoundError:
    print(f"""Error: Piper voice model file not found. Please ensure '{chosen_piper_voice}' is in the correct directory.""", file=sys.stderr)
    voice_model = None
except Exception as e:
    print(f"An unexpected error occurred while loading the Piper model: {e}", file=sys.stderr)
    voice_model = None

""" Load the Vosk speech recognition model.
Here we use a relatively small model. You can download a larger, much more accurate speech recognition model.
"""
sample_rate = 16000
try:
    speech_model = Model(model_name="vosk-model-en-us-0.22-lgraph")
except Exception as e:
    print(f"Error loading Vosk model: {e}. Please ensure the model is downloaded and accessible.", file=sys.stderr)
    speech_model = None
    speech_recognizer = None

if speech_model:
    try:
        speech_recognizer = KaldiRecognizer(speech_model, sample_rate)
        speech_recognizer.SetWords(True)
    except Exception as e:
        print(f"Error creating Vosk recognizer: {e}", file=sys.stderr)
        speech_recognizer = None

# Initialization of the LLM model and tokenizer.
# You can choose a different model. Remember that it may have different requirements.
llm_model_name = "Gensyn/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_name, pad_token_id=tokenizer.eos_token_id
)
llm_model.eval()

### Testing Models

It is recommended to test the microphone, speech recognition module, and speech synthesizer. To do this, run the code below. Click **Record** button, speak, then finish by clicking **Stop**. When running for the first time, allow the browser access to the microphone and run the code again.

In [ ]:
from IPython.display import Javascript, Audio, display
from google.colab.output import eval_js

# A simple Javascript code that will enable the recording of a signal from the microphone and encode it into an ASCII text format.
js = Javascript('''async function recordAudio() {
    const div = document.createElement('div');
    const startRecord = document.createElement('button');
    const stopRecord = document.createElement('button');

    startRecord.textContent = 'Record';
    stopRecord.textContent = 'Stop';

    document.body.appendChild(div);
    div.appendChild(startRecord);

    const stream = await navigator.mediaDevices.getUserMedia({audio:true});
    let audioRecorder = new MediaRecorder(stream);

    await new Promise((resolve) => startRecord.onclick = resolve);
    startRecord.replaceWith(stopRecord);
    audioRecorder.start();

    await new Promise((resolve) => stopRecord.onclick = resolve);
    audioRecorder.stop();
    let recData = await new Promise((resolve) => audioRecorder.ondataavailable = resolve);
    let arrBuff = await recData.data.arrayBuffer();
    stream.getAudioTracks()[0].stop();
    div.remove();

    let binaryString = '';
    let bytes = new Uint8Array(arrBuff);
    bytes.forEach((byte) => { binaryString += String.fromCharCode(byte) });

    const url = URL.createObjectURL(recData.data);
    const player = document.createElement('audio');
    player.controls = true;
    player.src = url;
    document.body.appendChild(player);

    return btoa(binaryString);
}
''')

# Decoding the text format into a standard wave binary format.
def get_audio(data):
    if data is not None:
        try:
            binary = b64decode(data)
        except:
            print("Probably microphone is not allowed.")
        finally:
            process = (ffmpeg
            .input('pipe:0')
            .output('-', format='s16le', acodec='pcm_s16le', ac=1, ar='16k')
            .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
            )
            output, err = process.communicate(input=binary)
            return output

# Converting spoken audio into written text.
def transcribe(data):
    audio = get_audio(data)
    if audio is not None:
        speech_recognizer.AcceptWaveform(audio)
        result = json.loads(speech_recognizer.FinalResult())
        recognized_text = result['text']
        if recognized_text:
            recognized_text = recognized_text.capitalize() + "."

        return recognized_text
    else:
        return ""


display(js)
obj = eval_js('recordAudio({})')
transcription = transcribe(obj)

print(f"Recognized speech: {transcription}")

for audio_chunk in voice_model.synthesize(transcription):
    display(Audio(audio_chunk.audio_int16_array, autoplay=True, rate=audio_chunk.sample_rate))

### Chat Application Frontend

We will import the user interface components and JavaScript functions that manage voice recording and communication with the Python backend in the Colab environment as a ready-made solution. It is an HTML document with an embedded stylesheet and JavaScript script.

In [10]:
voiceLLmFrontend = VoiceLLMChatFrontend_Colab(
    # Setting up our assistant's logo.
    assistantAvatarSrc = "https://qwenlm.github.io/img/logo.png",
    # For the user, let this be the Golab logo.
    userAvatarSrc = "https://colab.research.google.com/img/colab_favicon_256px.png"
    )

# Static HTML document with the application's interface.
llmChatFrontend = voiceLLmFrontend.getDocument()
# # You can also preview the content of the document.
# print(llmChatFrontend)

The downloaded version of the frontend will use five functions called from the JavaScript code. Below is the code implementing these functions.

### Chat Application

In [9]:
class LLMChatApp:
    """Main application class for the voice-enabled LLM chat."""
    def __init__(self, llm_model, tokenizer, voice_model, speech_recognizer):
        """Initializes the LLMChatApp with required models and components."""
        self.output_lock = threading.Lock()
        # Output widget to display messages and recognized text
        self.app_output_widget = widgets.Output()

        # Preparing application's backend
        self.app = VoiceLLMChatBackend(llm_model, tokenizer, voice_model, speech_recognizer)
        # Initialization of LLM model parameters.
        self.app.set_model_parameters(llm_model_temperature, llm_model_max_tokens, llm_model_top_k, llm_model_top_p, locale="en")
        self.app.set_system_message(llm_model_system_message)

        # # If the application does not appear to function as intended, enable this flag.
        # self.app.should_print_logs = True

        self.initialized = self.app.initialized

    def new_chat(self):
        """Starts a new chat session."""
        self.app.start_new_chat()
        return IPython.display.JSON({"response": "new chat created"})

    def send_prompt(self, prompt):
        """Sends a user prompt to the LLM."""
        self.app.send_prompt(prompt)
        return IPython.display.JSON({"response": "New prompt sent"})

    def fetch_data(self):
        """Fetches completed data chunks from the LLM chat backend."""
        try:
            data = self.app.get_completed_data_chunk()
            if data is not None:
                display_sentence, encoded_audio = data
                result = {
                    "resp": display_sentence,
                    "finish": "false"
                }
                if encoded_audio != "":
                    result["audio"] = encoded_audio

                return IPython.display.JSON(
                                result
                        )
            else:
                return IPython.display.JSON(
                {"resp": "", "finish": "true", "context": str(self.app.get_context_load())}
            )
        except Exception as e:
            print(f"Error in  fetch_data: {e}")

    def interrupt_response(self):
        """Interrupts the LLM's response generation."""
        self.app.interrupt_response()
        while self.app.is_model_working:
            time.sleep(0.1)
        response = self.app.get_last_response()
        return IPython.display.JSON(
            {
                "resp": response,
                "finish": "true",
                "context": str(self.app.get_context_load()),
            })

    def transcribe(self, data):
        """Transcribes audio data using the speech recognizer."""
        transcription = self.app.transcribe(data)
        return IPython.display.JSON({"result": transcription})

    def start_application(self):
        """Starts the main application logic."""
        self.app.start()

    def stop_application(self):
        """Stops the main application logic."""
        self.app.stop()

    def register_callbacks(self):
        """Registers the class methods as Colab output callbacks."""
        output.register_callback("notebook.new_chat", self.new_chat)
        output.register_callback("notebook.fetch_data", self.fetch_data)
        output.register_callback("notebook.transcribe", self.transcribe)
        output.register_callback("notebook.interrupt_response", self.interrupt_response)
        output.register_callback("notebook.send_prompt", self.send_prompt)


### Application Initialization

In [11]:
# Instantiate the new class
app_instance = LLMChatApp(llm_model, tokenizer, voice_model, speech_recognizer)

# A static HTML document that will allow for generating the application's interface.
app_instance.register_callbacks()

### Starting The Application

You need to allow your browser to use your microphone. When you first launch the application, it may be necessary to restart the code below. The first transcription takes a little longer due to the initialization of the speech recognition model.

In [ ]:
if app_instance.initialized:
    app_instance.app_output_widget.outputs = []
    display(app_instance.app_output_widget)

    app_instance.start_application()

    app_instance.app_output_widget.append_display_data(HTML(llmChatFrontend))
else:
    print("initialization failed")
    # Set the 'should_print_logs' flag to 'True', reload the application, and check the logs.

### Stopping The Application

To gracefully stop the application and release resources, uncomment and run the code cell below.

In [ ]:
# app_instance.stop_application()